<a href="https://colab.research.google.com/github/Thotasaiteja2004/Ai-lab/blob/main/assignment-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# Part 1: Import Required Libraries and Define Game Class
import collections
import random
import math

# Game Class Definition
class Game:
    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

In [35]:
# Part 2: Implementing the Game Strategy Algorithms
# Minimax Search Algorithm
def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""
    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)

# Alpha-Beta Search Algorithm
def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning."""
    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -math.inf, math.inf)

In [39]:
# Part 3: Implementing Tic Tac Toe Game
class Board(collections.defaultdict):
    """A board has the player to move, a cached utility value, and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""
    empty = '.'
    off = '#'

    def __init__(self, width=3, height=3, to_move=None, utility=0, **kwds):
        self.width = width
        self.height = height
        self.to_move = to_move
        self.utility = utility  # Initialize utility as an attribute
        super().__init__(lambda: self.empty, **kwds)
      #  self.update(width=width, height=height, to_move=to_move, utility=utility, **kwds)

    def new(self, changes: dict, **kwds) -> 'Board':
        """Given a dict of {(x, y): contents} changes, return a new Board with the changes."""
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def missing(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off

    def _hash_(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def _repr_(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'


class TicTacToe(Game):
    def _init_(self, height=3, width=3, k=3):
        self.k = k  # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = self.k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board):
        print(board)

    def k_in_row(self, board, player, square, k):
        """True if player has k pieces in a line through square."""
        def in_row(x, y, dx, dy):
            # Check if the coordinates are within the board's boundaries
            if 0 <= x < board.width and 0 <= y < board.height:
                return 0 if board.get((x, y), '.') != player else 1 + in_row(x + dx, y + dy, dx, dy)
            else:
                # If the coordinates are out of bounds, stop the recursion and return 0
                return 0

        return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
                   for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

SyntaxError: incomplete input (<ipython-input-39-7a70f1d52220>, line 74)

In [42]:
# Part 3: Implementing Tic Tac Toe Game
class Board(collections.defaultdict):
    """A board has the player to move, a cached utility value, and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""
    empty = '.'
    off = '#'

    def __init__(self, width=3, height=3, to_move=None, utility=0, **kwds):
        self.width = width
        self.height = height
        self.to_move = to_move
        self.utility = utility  # Initialize utility as an attribute
        super().__init__(lambda: self.empty, **kwds)
      #  self.update(width=width, height=height, to_move=to_move, utility=utility, **kwds)

    def new(self, changes: dict, **kwds) -> 'Board':
        """Given a dict of {(x, y): contents} changes, return a new Board with the changes."""
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def missing(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off

    def __hash__(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'


class TicTacToe(Game):
    def __init__(self, height=3, width=3, k=3):
        super().__init__()  # Call the superclass's __init__ method
        self.k = k  # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = self.k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board):
        print(board)

    def k_in_row(self, board, player, square, k):
        """True if player has k pieces in a line through square."""
        def in_row(x, y, dx, dy):
            # Check if the coordinates are within the board's boundaries
            if 0 <= x < board.width and 0 <= y < board.height:
                return 0 if board.get((x, y), '.') != player else 1 + in_row(x + dx, y + dy, dx, dy)
            else:
                # If the coordinates are out of bounds, stop the recursion and return 0
                return 0

        return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
                   for (dx, dy) in ((0, 1), (1, 0), (1,1),(1,-1)))

In [43]:
# Part 4: Evaluate the Game Strategies
def random_player(game, state):
    """Return a random legal move for the current player."""
    return random.choice(list(game.actions(state)))

def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game."""
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose:
            print('Player', player, 'move:', move)
            print(state)
    return state

if __name__ == "__main__":
    print("Playing Tic Tac Toe:")
    game_result = play_game(
        TicTacToe(),
        dict(X=random_player, O=lambda game, state: alphabeta_search(game, state)[1]),
        verbose=True
    )
    print("Game result utility for X:", game_result.utility)

Playing Tic Tac Toe:
Player X move: (2, 0)
. . X
. . .
. . .

Game result utility for X: 0
